# **Introduction to Artificial Intelligence - MO416A**
**UNIVERSITY OF CAMPINAS**



This work was completed by the following members:



*   Aissa Hadj - 265189
*   Lucas Zanco Ladeira - 188951
*   Matheus Ferraroni - 212142
*   Maria Vitória Rodrigues Oliveira - 262884
*   Oscar Ciceri - 164786

The original code of the project is located on a [repository inside Github](https://github.com/lucaslzl/ga_ia_p2) and the video showing the search strategies working is on [youtube](https://youtube.com). 



# 1 - Introduction



The problem that will be tackled in this project is Feature Selection. The goal is to obtain the subset of available features in a dataset that improves model performance by increasing its accuracy and decreasing its error rate. With the presence of irrelevant features in the dataset, more processing and memory requirements are necessary, thus wasting computing resources. To better understand the possible impacts of feature selection, we could cite the following pros: 

- Reducing Overfitting (nao concordo com esse ponto)
- Improving the model Accuracy
- Reducing Training Time
- removing some features with redundant information
- saving time and costs by collecting fewer features when deploying the model in production
- requiring less computing resources such as memory
- reducing the delay of the model in generating results

Feature Selection may be done manually or automatically. Some manual techniques include: univariate selection, feature importance, and the correlation matrix. The objective of the Univariate selection method is to statistically describe the relation between each feature and the target. Also, feature importance generates a score for each feature in order to rank it. For instance, Decision Tree algorithms may rank features according to Gini impurity tests. Finally, the Correlation Matrix shows the correlation between pairs of features so that the features with a high correlation could be removed. The literature presents the usage of optimization techniques to automatically find the best (or a quite good) subset of features. Some of the methods include:

- <b>Exhaustive search</b>
- <b>Simulated Annealing</b>
- <b>Transformation Graph</b>
- <b>Genetic Algorithms</b>

<b>Exhaustive Search</b> is not an optimization technique, but it is worth to be mentioned as its computational complexity is $O(2^n)$. This technique tries every possible subset of features in order to find the best one. Due to its computational complexity, this technique is not practical in most cases. <b>Simulated Annealing</b> is a metaheuristic for complex nonlinear optimization problems and is analogous to the simulation of the annealing of solids. The analogy pairs are as follows: feasible solution (state), cost (energy), optimal solution (ground state), local search (rapid quenching), simulated annealing (careful annealing). On the other hand, <b>Transformation Graph</b> is a strategy that utilizes a tree-like structure to generate possible solutions. First, $n$ solutions are generated by removing at each one distinct feature. Second, all the solutions are evaluated. Third, the best solution is chosen and $n-1$ are generated by removing each feature yet not removed. That strategy goes on considering a budget. The issue of this strategy is that it requires a substantial amount of memory. Finally, <b>Genetic Algorithm</b> is inspired by genetics (DNA) to search through solutions. Its process can be described with the following steps: (1) it generates a population considering variations of the DNA, (2) then ranks the population according to some score, it applies some forms of mutations and other transformations to the DNA at each generation, and finally, (3) it iterates through the previous steps until it reaches a stopping rule. The stopping rule may be the amount of generations produced or a target solution was reached. In the current project, we decided to further explore this strategy by applying it to a specific modelization problem. More details will be given in the next sections.

This report is structured as follows. In section 2, we describe the implementation of the main parts of the genetic algorithm. Then, in section 3, we discuss the methodology we followed to undertake this project. Finally, in section 4, we do a detailed analysis of the results.  

# 2 - Genetic Algorithm

We developed 2 classes to control the operation of the genetic algorithm. The classes are 'element' and 'GeneticAlgorithm' and can be seen in https://github.com/lucaslzl/ga_ia_p2/blob/master/GA.py. In this notebook, the class 'element' will be explained, as well as the main methods of the class 'GeneticAlgorithm'.

Below, the entire class 'element' is presented. It is possible to see that this class is only responsible for managing the id, the generation, the genome and the score of each element of the population for every generation. Saving this attributes in the same place can be useful for different approaches during the implementation and use of the methods on the genetic algorithm. It is also possible to save the parents of each element and traceback how each element was formed during the evolution.

In [0]:
class element:

    def __init__(self, idd, geracao, genome):
        self.idd = idd
        self.geracao = geracao
        self.genome = genome
        self.score = None


    def __repr__(self):
        return "(id="+str(self.idd)+",geracao="+str(self.geracao)+",score="+str(self.score)+")"

The genetic algorithm implemented is very generic, this means that it can be applied easily to different problems. Using this kind of generic implementation makes it practical to override the functions responsible to generate a random genome, to mutate and perform the fitness calculation.

The method 'create_initial_population' is called once the genetic algorithm is launched. This method is responsible for creating the individuals of the initial population and add them to the pool until the required population size is reached. To create the genome, the function 'random_genome', that was overwritten before, is called for each element.

In our problem, we define the genome as an array of 0's and 1's as elements with a length equal to the amount of features in the dataset. This kind of approach used in the genome allows us to decode the genome as:

- Every bit of the genome corresponds to a feature in the dataset
- If the bit is 1: the feature corresponding to that bit is used
- If the bit is 0: the feature corresponding to that bit isn't used

In [0]:
def create_initial_population(self):
    for _ in range(self.population_size):
        self.population.append(element(self.elements_created, 0, self.random_genome()))
        self.elements_created += 1


def random_genome():
    return np.random.randint(low=0,high=2,size=len(df.columns),dtype=int)

The method 'run' is where the main loop of the genetic algorithm is executed. The steps are:

1. Check the stop criteria
2. Calculate the score for the current population
3. Sort the population according to the fitness value
4. Update the solution if a better result was found
5. Save the log
6. If set, part of the worst part the of population can be discarted (This is not being used in the solutions found for this work)
7. Create a new population

In [0]:
def run(self):

    while self.check_stop():
        self.calculate_score() 
        self.population.sort(key=lambda x: x.score, reverse=True) 

        if self.best_element_total==None or self.population[0].score > self.best_element_total.score: 
            self.best_element_total = self.population[0]

        self.do_log()

        if self.cut_half_population: 
            self.population = self.population[0:len(self.population)//2] 

        self.new_population()

        self.iteration_counter +=1

    return self.best_element_total

The creation of a new population is implemented to be the same independently to how the selection of the parents is made. In order to achieve this, we create an array of probability that implements the rules for this selection based on equal chance of each element or by roulette, where elements with higher fitness have higher chances to be selected.

The crossover method receives the parent's genome and returns a new genome based on its genome.
This new genome is inserted into the new element and this new element is added to the new pool.

If we are recreating the entire population, this proccess repeats till the new population has the size of the population size limit. If we are replicating a percentage of the best elements, the amount of best elements being replicated is reduced in this proccess and they are replicated after this process.

In [0]:
def new_population(self):

    probs = self.get_probs()
    newPop = []
    best_replicator = int(self.population_size*self.replicate_best)

    while len(newPop)<self.population_size-best_replicator:
        parents = np.random.choice(self.population,size=2,p=probs) 

        if parents[0].score<parents[1].score: 
            parents = parents[::-1] 

        new_element = element(self.elements_created, self.iteration_counter, self.crossover(parents[0].genome, parents[1].genome))

        new_element.genome = self.active_mutate(new_element.genome)
        newPop.append(new_element)
        self.elements_created += 1

    for i in range(best_replicator):
        newPop.append(self.population[i])

    self.population = newPop

In order to define the probability of being selected as a parent, we implement 3 methods: "get_probs", "probs_equal" and "probs_roulette".

The "get_probs" just checks what kind of probability function must be used and calls the right one.
The "probs_equal" returns an array of probability where every element has the same chance of being selected.
The "probs_roullete" adds the fitness of each element to the array, sums their total and divides the array by this sum. The result is an array where the best solutions have higher chances of being selected.

In [0]:
def get_probs(self):
    if self.probs_type == 0:
        return self.probs_roulette()
    elif self.probs_type == 1:
        return self.probs_equal()


def probs_equal(self):
    return [1/len(self.population)]*len(self.population)


def probs_roulette(self):
    probs = [0]*len(self.population) 
    for i in range(len(probs)):
        probs[i] = self.population[i].score
    div = sum(probs)

    if div!=0:
        for i in range(len(probs)):
            probs[i] /= div
    else: 
        probs = self.probs_equal()
    return probs

The method "check_stop" is responsible for checking which stop criteria must be used. The stop criteria can be used in 3 different ways.

The "stop_criteria_iteration" just returns "True" if a minimum amount of iterations has been attained.
The "stop_criteria_score" just returns "True" if any solution for a given generation achieved a minimum score.
The "stop_criteria_double" is a mix of the previous 2 methods.

In [0]:
def check_stop(self):
    if self.stop_criteria_type==0:
        return self.stop_criteria_double()
    elif self.stop_criteria_type==1:
        return self.stop_criteria_iteration()
    elif self.stop_criteria_type==2:
        return self.stop_criteria_score()

def stop_criteria_double(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return self.iteration_counter<self.iteration_limit or s>=self.max_possible_score

def stop_criteria_iteration(self):
    return self.iteration_counter<self.iteration_limit

def stop_criteria_score(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return s>=self.max_possible_score

We implemented 4 different crossovers and 1 function to decide which one must be used. The function "crossover" just receives the genome of 2 parents and calls the right genome function.

The "crossover_rate_selection" iterates the genome of both parents and selects from which one the bit must be selected. In order to define which one to select, this function checks a percentage that was defined previously. This means that the resulting genome can be 80% from one parent and 20% from the other, where the 80% comes from a parent with the highest score.

The "crossover_uniform" just selects the bits from the parent with a chance close to 50%/50% from selection from each parent.

The "crossover_single_point" defines a random point in the middle of the genome and picks the first part from parentA and the second part from the parentB.

The "crossover_two_point" defines 2 random points in the middle of the genome and cocatenates a part of the genomeA, a part from genomeB and a part from the genomeA using the points defined.

In [0]:
def crossover(self, genA, genB):
    if self.crossover_type==0:
        return self.crossover_uniform(genA, genB)
    elif self.crossover_type==1:
        return self.crossover_single_point(genA, genB)
    elif self.crossover_type==2:
        return self.crossover_two_point(genA, genB)
    elif self.crossover_type==3:
        return self.crossover_rate_selection(genA, genB)

def crossover_rate_selection(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<self.crossover_rate:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_uniform(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<0.5:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_single_point(self, genA, genB):
    p = np.random.randint(low=1,high=len(genA)-1) 
    return np.append(genA[0:p],genB[p:])


def crossover_two_point(self, genA, genB):
    c1 = c2 = np.random.randint(low=0,high=len(genA)) 
    while c2==c1: 
        c2 = np.random.randint(low=0,high=len(genA))

    if c1>c2: 
        c1, c2 = c2,c1

    new = np.append(np.append(genA[0:c1],genB[c1:c2]),genA[c2:]) 
    return new

The method "calculate_score" was implemented to be used synchronously or asynchronously and the user can define how to execute it before the start of the main loop.

This function just passes the genome of each element to a function that returns its fitness. In our case, we call the function "evaluate" that is responsible to decode the genome to use or not the features in the dataset and to check how good these features perform.

In [0]:
def calculate_score(self):
    if self.use_threads: 

        threads_running = []
        for e in self.population:
            x = threading.Thread(target=self.thread_evaluate, args=(e,))
            x.start()
            threads_running.append(x)

        for i in range(len(threads_running)):
            threads_running[i].join()

    else: 
        for e in self.population:
            e.score = self.evaluate(e.genome)

def thread_evaluate(self, e):
    e.score = self.evaluate(e.genome)
    

def evaluate(genome):
    bool_genome = list(map(bool, genome))
    return model.evaluate(df.loc[:, bool_genome].copy(), target)

The mutation method is called "active_mutate" and receives a single genome. This method iterates through the entire genome and creates a random value, if this value is smaller than the one set in the initialization, then a mutation is started on that index.

We implemented 2 different mutations for this project:

- mutate1: This method implements the generative mutation, which randomly changes a gene.  The genes have binary values; thus, the selected gene changes the allele value for his complement.


- mutate2: This method implements the sequence swap combined with a generative mutation. First, a random position of the gene on a chromosome is selected. The genes located after this position are move to the beginning on the chromosome, and genes located before are move to the last. Moreover, the generative mutations technique is employed in the new chromosome. 

In [0]:
def active_mutate(self,gen):
    if self.mutation_rate<=0: 
        return gen
    for i in range(len(gen)): 
        if np.random.random()<self.mutation_rate: 
            gen = self.mutate1(i, gen) 
    return gen


def mutate1(index, genome):
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome


def mutate2(index, genome):
    aux = []
    for i in range(len(genome)):
        if i <= index:
            aux.append(genome[i])
        else:
            aux.insert(0, genome[i])
    genome = aux
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome

# 3 - Methodology

In this project, we apply the genetic algorithm to the frequent challenge of building machine learning models applied to classification problems. The term "target" is defined as the variable that the machine learning model must predict and classify as one class among a choice of k classes. In order to build the model, we use "features" that are defined as the modalities that are observed with the assumption that they are relevant in making the prediction about the target. The objective in our work is thus to measure the importance of each feature in helping to derive the value of the target, relative to the other available features. The end goal is thus to perform a feature selection by choosing a final subset of features used by the machine learning model to make classification decisions.

To help us determine the impact of genetic algorithm in the feature selection, we combine this method with the decision tree technique applied to tabular data. 

The following subsections describe the decision tree model in general, the tabular data we chose, and the metrics we measure to evaluate the genetic algorithm in feature selection.







## 3.1 - Decision Tree

A decision tree is a flowchart-like structure in which each internal node represents a test on a feature. In the figure below, "is the income over or below $30,000?" is a test performed on the feature called "Income" for instance. Each branch represents the outcome of the test, and each leaf node represents a class label. A decision about the class predicted is taken after performing a series of test on features, until reaching a leaf node. The paths from root to leaf represent classification rules.


<img src='https://raw.githubusercontent.com/lucaslzl/ga_ia_p2/master/images%20for%20report/decision%20tree%20example.png'>



## 3.2 - Tabular Data

We use the term "tabular data" to designate datasets composed of features that could take numerical values (for instance the income amount of a client with a checking account in a bank institution) or categorical values ("Yes" or "No" values describing if a client has a membership in a mileage program from an airline company for instance). 

We chose to use only tabular data for practical reasons. They are relatively easy to preprocess and clean before feeding them to a decision tree for model building. This step is termed as the "training" part. Also, it will make it easier for us to interpret the results of feature selection performed by the genetic algorithm.

In order for our project to be as close as possible to reality, the majority of the datasets we use come from the Kaggle website. Kaggle is an organization where people compete in building the best machine learning model for real world problems. 

The number of features from each dataset varies from 10 to over 500. That will help us evaluate the computing resources needed by the genetic algorithm applied to the decision tree technique, and its practicality in applying it to real world. In particular, as we will see in more detail in the results section, as we increase the number of features collected in a given dataset, the number of observations needed, to build a relatively "good" machine learning model, increases at an exponential rate. This issue is termed as "the curse of dimensionnality". We mention here the importance in varying the number of features since it impacts tremendously the amount of resources needed by the genetic algorithm in terms of time and computing power.

The description of the datasets we selected are provided as links to the sources where the datasets are encountered. The full list is provided in the last section of this report called "Source". In addition, the datasets can consulted in the following folder: https://github.com/lucaslzl/ga_ia_p2/tree/master/data


## 3.3 - Metrics

# 4 - Results

# 5 - Conclusions

Points to discuss:
1. decision tree model is relatively easy to implement and can be a first choice for machine learning model building. If the results are limited, then more complex models can be explored using also genetic algorithm. 
2. decision tree can be used to filter the features for building more complex models and discarding features giving redundant information about the target. Thus helping saving time along the way of building heavy machine learning models.
3. as we increase the number of features (talk about the "curse of dimensionality"), genetic algorithm requires increasing time. But it's a step that is executed only once.
4. genetic algorithm was applied to classification problems, but can be applied to regression and to features not necessarily tabular. Other types of feaures could be images, texts, etc.
5. comparison with Principa Component Analysis.



# Sources

*Datasets used in the project:*